# Lab 2 PySpark:

### Machine Learning

#### But first, let's do some SQL :)

**First, install and import PySPark and SparkSession**

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 26 kB/s 
     |████████████████████████████████| 198 kB 53.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f3986616dd4e6f456f2041a3a889cf94de781199670a39759261ec341b132331
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn

In [ ]:
spark = (SparkSession
.builder
.appName("Lab2")
.getOrCreate())
sc = spark.sparkContext

**Now, download the dataset**

In [ ]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 122MB/s]


**Create a SparkSession object and name the app "Lab2"**

**1. Read the file PatientInfo.csv into a dataframe**

In [ ]:
df_pth=spark.read.csv('/content/PatientInfo.csv',header=True)

**Show the first 5 lines of the dataframe**

In [ ]:
df_pth.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

**Now do the same but using SQL select statement**

1. Create a temporary view (table) called patients

In [ ]:
df_pth.createOrReplaceTempView('patients')


2. Use SELECT statement to select all columns from the dataframe

In [ ]:
spark.sql("select * from patients").show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

3. Limit the output to only 5 rows *using SQL commands*

In [ ]:
spark.sql("select * from patients limit 5").show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|   202

4. Select the count of males and females in the dataset

In [ ]:
spark.sql("SELECT SUM(CASE WHEN UPPER(sex) = 'MALE' THEN 1 ELSE 0 END) AS Male,\
SUM(CASE WHEN UPPER(sex) = 'FEMALE' THEN 1 ELSE 0 END) AS Female\
  FROM patients").show()

+----+------+
|Male|Female|
+----+------+
|1825|  2218|
+----+------+



5. Select the count of males and females *as percentage* (how many percent of the data are males and how many are females?)

In [ ]:
spark.sql("SELECT  (SUM(CASE WHEN UPPER(sex) = 'MALE' THEN 1 ELSE 0 END)/COUNT(sex)) AS Male,\
(SUM(CASE WHEN UPPER(sex) = 'FEMALE' THEN 1 ELSE 0 END)/COUNT(sex)) AS Female\
  FROM patients").show()

+------------------+------------------+
|              Male|            Female|
+------------------+------------------+
|0.4513974771209498|0.5486025228790502|
+------------------+------------------+



6. How many people did survive, and how many didn't?

In [ ]:
spark.sql("SELECT SUM(CASE WHEN UPPER(state) = 'released' THEN 1 ELSE 0 END) AS rs,\
SUM(CASE WHEN UPPER(state) != 'deceased' THEN 1 ELSE 0 END) AS de\
  FROM patients").show()

+---+----+
| rs|  de|
+---+----+
|  0|5165|
+---+----+



Now, let's perform some preprocessing using SQL:

1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [ ]:
df2=spark.sql(" SELECT sex, province, state,  CAST(REPLACE(REPLACE(age,'s', ''), 's', '')  as double) AGE FROM  patients WHERE age IS NOT NULL")

Now view the new dataframe to make sure everything is alright

In [ ]:
df2.show()

+------+--------+--------+----+
|   sex|province|   state| AGE|
+------+--------+--------+----+
|  male|   Seoul|released|50.0|
|  male|   Seoul|released|30.0|
|  male|   Seoul|released|50.0|
|  male|   Seoul|released|20.0|
|female|   Seoul|released|20.0|
|female|   Seoul|released|50.0|
|  male|   Seoul|released|20.0|
|  male|   Seoul|released|20.0|
|  male|   Seoul|released|30.0|
|female|   Seoul|released|60.0|
|female|   Seoul|released|50.0|
|  male|   Seoul|released|20.0|
|  male|   Seoul|deceased|80.0|
|female|   Seoul|released|60.0|
|  male|   Seoul|released|70.0|
|  male|   Seoul|released|70.0|
|  male|   Seoul|released|70.0|
|  male|   Seoul|released|20.0|
|female|   Seoul|released|70.0|
|female|   Seoul|released|70.0|
+------+--------+--------+----+
only showing top 20 rows



**Now, let's get back to spark operations**

Please copy the following operations from your solution in Lab 1
___

Add a **is_dead** column if patient state is not released then it should yield true, else then False

In [ ]:
df3 = df2.withColumn("is_dead", fn.when(df2.state != "released",1).otherwise(0))
df3.show()

+------+--------+--------+----+-------+
|   sex|province|   state| AGE|is_dead|
+------+--------+--------+----+-------+
|  male|   Seoul|released|50.0|      0|
|  male|   Seoul|released|30.0|      0|
|  male|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|female|   Seoul|released|20.0|      0|
|female|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|released|30.0|      0|
|female|   Seoul|released|60.0|      0|
|female|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|deceased|80.0|      1|
|female|   Seoul|released|60.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|20.0|      0|
|female|   Seoul|released|70.0|      0|
|female|   Seoul|released|70.0|      0|
+------+--------+--------+----+-------+
only showing top 20 rows



**Please split the data into train and test dataframes**

*Ratio: 80:20 - Seed=42*

In [ ]:
trainDF, testDF = df3.randomSplit([0.8,0.2],seed=42)

In [ ]:
testDF.show(5)

+------+--------+--------+----+-------+
|   sex|province|   state| AGE|is_dead|
+------+--------+--------+----+-------+
|  null|  Sejong|isolated|40.0|      1|
|female|   Busan|isolated|20.0|      1|
|female|   Busan|isolated|30.0|      1|
|female|   Busan|isolated|70.0|      1|
|female|   Busan|released|10.0|      0|
+------+--------+--------+----+-------+
only showing top 5 rows



In [ ]:
mn = df3.select('age').agg(fn.mean(fn.col('age')).alias('AvgAge'))

In [ ]:
mn =mn.toPandas()

In [ ]:
mn=list(mn['AvgAge'])

**Now, let's import RandomForestClassifier and start our ML pipeline**

In [ ]:
from pyspark.mllib.tree import RandomForest
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer

**Create a pipeline that contains the following stages:**

- Imputer: impute the null values in `age` column to the mean value
- StringIndexer: convert `sex` to `is_male` and `province` to `province_index` as numerical values
- OneHotEncoder: perform one hot encoding on both `is_male` and -province_index`
- VectorAssembler: assemble feature vector from the following columns: `'age', 'is_male', 'province_index'`
- RandomForestClassifier: final estimator

+------+--------+--------+----+-------+
|   sex|province|   state| AGE|is_dead|
+------+--------+--------+----+-------+
|  male|   Seoul|released|50.0|      0|
|  male|   Seoul|released|30.0|      0|
|  male|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|female|   Seoul|released|20.0|      0|
|female|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|released|30.0|      0|
|female|   Seoul|released|60.0|      0|
|female|   Seoul|released|50.0|      0|
|  male|   Seoul|released|20.0|      0|
|  male|   Seoul|deceased|80.0|      1|
|female|   Seoul|released|60.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|70.0|      0|
|  male|   Seoul|released|20.0|      0|
|female|   Seoul|released|70.0|      0|
|female|   Seoul|released|70.0|      0|
+------+--------+--------+----+-------+
only showing top 20 rows



In [ ]:
cols=df3.dtypes

In [ ]:
categoricalCol=[f for(f,d) in cols if d=='string']
categoricalCol


['sex', 'province', 'state']

In [ ]:
OutputCol_index = [f + '_indx'for f in categoricalCol ]
OutputCol_index

['sex_indx', 'province_indx', 'state_indx']

In [ ]:
OHE_output_col = [f + '_OHE'for f in categoricalCol ]
OHE_output_col 

['sex_OHE', 'province_OHE', 'state_OHE']

In [ ]:
numericalCol = [f for(f,d) in cols if( (d=='double' or d=='int') and (f !='crew'))]
numericalCol

['AGE', 'is_dead']

In [ ]:
ageImput =Imputer(strategy=mn[0], inputCols=['AGE'], outputCols=['AGEImpute'])

In [ ]:
assemblerInputs = OHE_output_col + numericalCol

In [ ]:
stringIndex = StringIndexer(inputCols=categoricalCol, outputCols=OutputCol_index ,handleInvalid='skip')

OHE=OneHotEncoder(inputCols=OutputCol_index , outputCols=OHE_output_col)

vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol='features',labelCol='crew',predictionCol='prediction')

Fit the pipeline to the train dataframe

In [ ]:
stgs = [stringIndex,OHE,vecAssembler,rf]

Now transform the test DF to get predictions

Show the final predictions DF

In [ ]:
pred.select('features','is_dead','prediction').show(5,truncate=False)

**Model Evaluation**

Now let's evaluate our model! Let's get the accuracy of our model

Excellent! Now let's generate the confusion matrix of our predictions

*Hint: we can use `scikit-learn`'s `classification_report`. You will need to transform the predictions into pandas DF first*